In [ ]:
from google.colab import drive
drive.mount('drive')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import time as t
import datetime
import os
import shutil
from zipfile import ZipFile

In [ ]:
path = '/content/drive/MyDrive/Degree Project/dataset/csv_format/'
uni_path = '/content/drive/MyDrive/Degree Project/dataset/extracts/'
local_path = '/content/'
trip_df = pd.read_csv(path + 'trips.csv')
transfers_df = pd.read_csv(path + 'transfers.csv')
stops_df = pd.read_csv(path + 'stops.csv')
stoptimes_df = pd.read_csv(path + 'stop_times.csv')
shapes_df = pd.read_csv(path + 'shapes.csv')
routes_df = pd.read_csv(path + 'routes.csv')
feedinfo_df = pd.read_csv(path + 'feed_info.csv')
agency_df = pd.read_csv(path + 'agency.csv')
calendar_df = pd.read_csv(path + 'calendar_dates.csv')
rg_data = pd.read_csv(uni_path + 'road_grades.csv')

In [ ]:
stoptimes_df.head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,shape_dist_traveled,stop_headsign,pickup_type,drop_off_type,timepoint
0,132562:299107,15:50:00,15:50:00,1269,1,0,NaN,0,0,1
1,132562:299107,15:50:00,15:50:00,1270,2,185,NaN,0,0,0
2,132562:299107,15:51:00,15:51:00,1271,3,346,NaN,0,0,0
3,132562:299107,15:51:00,15:51:00,1272,4,578,NaN,0,0,0
4,132562:299107,15:52:00,15:52:00,1273,5,763,NaN,0,0,0


In [ ]:
def weird_division(n, d): # "weird_division" courtsey of stackoverflow
  return n/d if d else 0
trips = list(stoptimes_df.trip_id.unique())
routes = list(trip_df.route_id.unique())
FMT = '%H:%M:%S'
time_buffer = 30/3600.0 # adding a buffer ish value so that speed cannot be zero, given distance changed 
global_speed, global_acceleration  = [], []
avg_speed, avg_acceleration, number_of_stops = {}, {}, []
for trip in trips:
  dist_diff, times, time_diff, speed, speed_diff, acceleration = [], [], [], [], [], []
  temp_df = stoptimes_df.loc[stoptimes_df['trip_id'] == trip]
  temp_df['shape_dist_traveled'] = temp_df['shape_dist_traveled']*0.001 # convert to km
  dist_diff = list(temp_df['shape_dist_traveled'].diff()) # km
  nos = len(temp_df['stop_sequence'].tolist()) # number of stops for a trip
  for time in temp_df['arrival_time']:
    x = t.strptime(time, FMT)
    times.append((datetime.timedelta(hours=x.tm_hour,minutes=x.tm_min,seconds=x.tm_sec).total_seconds())/3600.0) # convert to hours
    number_of_stops.append(nos) # using the loop to append number of stops to df for calculating stop density later on
  time_diff = list(np.diff(times)) # hr
  time_diff.insert(0,0)
  time_diff[1:] = [time + time_buffer for time in time_diff[1:]]
  if len(dist_diff) == len(time_diff):
    for dist, time in zip(dist_diff, time_diff):
      speed.append(weird_division(dist,time)) #km/h
      global_speed.append(weird_division(dist,time)) # expected length = length of stoptimes_df
    speed_diff = list(np.diff(speed))
    speed_diff.insert(0,0)
    for spd, time in zip(speed_diff, time_diff):
      acceleration.append(weird_division(spd, time)) #km/hr^2
      global_acceleration.append(weird_division(spd, time)) # expected length = length of stoptimes_df
  avg_speed[trip] = (sum(speed)/len(speed))
  avg_acceleration[trip] = (sum(acceleration)/len(acceleration))
stoptimes_df['speed'] = pd.Series(global_speed).values
stoptimes_df['acceleration'] = pd.Series(global_acceleration).values
stoptimes_df['number_of_stops'] = pd.Series(number_of_stops).values

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [ ]:
date_format = '%Y%m%d'
stops, service_id, shape_id, route_id = [], [], [], []
for id in stoptimes_df['stop_id']:
  stops.append(stops_df.loc[stops_df['stop_id'] == id, 'stop_name'].iloc[0])
stoptimes_df['stop_headsign'] = pd.Series(stops).values
for trip in stoptimes_df['trip_id']:
  service_id.append(trip_df.loc[trip_df['trip_id'] == trip, 'service_id'].iloc[0])
  shape_id.append(trip_df.loc[trip_df['trip_id'] == trip, 'shape_id'].iloc[0])
  route_id.append(trip_df.loc[trip_df['trip_id'] == trip, 'route_id'].iloc[0])
stoptimes_df['service_id'] = pd.Series(service_id).values
stoptimes_df['shape_id'] = pd.Series(shape_id).values
stoptimes_df['route_id'] = pd.Series(route_id).values
stoptimes_df.to_csv('trip_data.csv', index = False)
shutil.move(local_path + 'trip_data.csv', uni_path + 'trip_data.csv')

'/content/drive/MyDrive/Degree Project/dataset/extracts/trip_data.csv'

In [ ]:
trips = list(stoptimes_df.trip_id.unique())
station_in_path, dist_from_station, time_at_station = {}, {}, {}
stations = ['City Hall Terminal', 'Waterfront Terminal']
for trip in trips:
  temp_df = stoptimes_df.loc[stoptimes_df['trip_id'] == trip]
  has_station = 'NA'
  d_fs = 0
  time_stamp = 'no_time'
  for stop in temp_df['stop_headsign'][1:]:
    if stop in stations:
      has_station = stop
      d_fs = temp_df.loc[temp_df['stop_headsign'] == stop, 'shape_dist_traveled'].iloc[0]
      time_stamp = temp_df.loc[temp_df['stop_headsign'] == stop, 'arrival_time'].iloc[0]
      break
    else:
      has_station = 'NA'
      d_fs = 0
      time_stamp = 'no_time'
  station_in_path[trip] = has_station # track if this trip has a station in it's path
  dist_from_station[trip] = d_fs # track the distance from station
  time_at_station[trip] = time_stamp # track time arrived at station

In [ ]:
trip_data = stoptimes_df.groupby(['trip_id'])
trip_data = trip_data.first().append(trip_data.last()).sort_index().reset_index().drop_duplicates()
trip_data = trip_data.sort_values(['trip_id', 'arrival_time'], ascending=True)
trip_data = trip_data.drop(['pickup_type', 'drop_off_type', 'timepoint'], axis=1)
trip_data.head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,shape_dist_traveled,stop_headsign,speed,acceleration,number_of_stops,service_id,shape_id,route_id
0,132374:304860,09:00:00,09:00:00,1423,1,0,Dawson & Regina,0.000,0.000,26,304860,3306,13
1,132374:304860,09:15:00,09:15:00,1121,26,7202,Waterfront Terminal,10.344,62.016,26,304860,3306,13
2,132375:305353,09:40:00,09:40:00,1423,1,0,Dawson & Regina,0.000,0.000,26,305353,3306,13
3,132375:305353,09:55:00,09:55:00,1121,26,7202,Waterfront Terminal,10.344,62.016,26,305353,3306,13
4,132376:304770,10:20:00,10:20:00,1423,1,0,Dawson & Regina,0.000,0.000,26,304770,3306,13


In [ ]:
route_data = {}
route_data['trip_id'] = trip_data['trip_id'][::2].tolist()
route_data['service_id'] = trip_data['service_id'][::2].tolist()
route_id, trip_headsign = [],[]
for trip in route_data['trip_id']:
  route_id.append(trip_df.loc[trip_df['trip_id'] == trip, 'route_id'].iloc[0])
  trip_headsign.append(trip_df.loc[trip_df['trip_id'] == trip, 'trip_headsign'].iloc[0])
route_data['trip_headsign'] = trip_headsign
route_data['route_id'] = route_id
route_data['shape_id'] = trip_data['shape_id'][::2].tolist()
route_data['origin'] = trip_data['stop_headsign'][::2].tolist()
route_data['destination'] = trip_data['stop_headsign'][1::2].tolist()
route_data['start_time'] = trip_data['arrival_time'][::2].tolist()
route_data['end_time'] = trip_data['departure_time'][1::2].tolist()
start_time, end_time, road_grade = [], [], []
for shape_id in route_data['shape_id']:
  road_grade.append(rg_data.loc[rg_data['shape_id'] == shape_id, 'road_grade(%)'].iloc[0])
route_data['road_grade'] = road_grade
for time in route_data['start_time']:
  x = t.strptime(time, FMT)
  minutes = (datetime.timedelta(hours=x.tm_hour,minutes=x.tm_min,seconds=x.tm_sec).total_seconds())/60
  start_time.append(minutes)
for time in route_data['end_time']:
  x = t.strptime(time, FMT)
  minutes = (datetime.timedelta(hours=x.tm_hour,minutes=x.tm_min,seconds=x.tm_sec).total_seconds())/60
  end_time.append(minutes)
total_time = [None] * len(start_time)
for i in range(len(start_time)):
  total_time[i] = (end_time[i] - start_time[i])
route_data['total_time'] = total_time
route_data['total_distance'] = trip_data['shape_dist_traveled'][1::2].tolist()
avg_speed_list, avg_acceleration_list, stop_density, has_station, distance_fs, time_as = [], [], [], [], [], []
for trip in route_data['trip_id']:
  avg_speed_list.append(avg_speed[trip])
  avg_acceleration_list.append(avg_acceleration[trip])
  has_station.append(station_in_path[trip])
  distance_fs.append(dist_from_station[trip])
  time_as.append(time_at_station[trip])
  stop_density.append((trip_data.loc[trip_data['trip_id'] == trip, 'number_of_stops'].iloc[1])\
                      /(trip_data.loc[trip_data['trip_id'] == trip, 'shape_dist_traveled'].iloc[1] * 0.001 )) # stop density = stops / km
route_data['avg_speed'] = avg_speed_list
route_data['avg_acceleration'] = avg_acceleration_list
route_data['stop_density'] = stop_density
route_data['has_station'] = has_station
route_data['dist_to_station'] = distance_fs
route_data['time_at_station'] = time_as

In [ ]:
route_data = pd.DataFrame.from_dict(route_data, orient='index').transpose()
route_data.head()

,trip_id,service_id,trip_headsign,route_id,shape_id,origin,destination,start_time,end_time,road_grade,total_time,total_distance,avg_speed,avg_acceleration,stop_density,has_station,dist_to_station,time_at_station
0,132374:304860,304860,JOHN-JUMBO to Waterfront,13,3306,Dawson & Regina,Waterfront Terminal,09:00:00,09:15:00,-3.02,15.0,7202,19.414769,432.637538,3.610108,Waterfront Terminal,7202,09:15:00
1,132375:305353,305353,JOHN-JUMBO to Waterfront,13,3306,Dawson & Regina,Waterfront Terminal,09:40:00,09:55:00,-3.02,15.0,7202,19.414769,432.637538,3.610108,Waterfront Terminal,7202,09:55:00
2,132376:304770,304770,JOHN-JUMBO to Waterfront,13,3306,Dawson & Regina,Waterfront Terminal,10:20:00,10:35:00,-3.02,15.0,7202,19.414769,432.637538,3.610108,Waterfront Terminal,7202,10:35:00
3,132377:305609,305609,JOHN-JUMBO to Waterfront,13,3306,Dawson & Regina,Waterfront Terminal,11:00:00,11:15:00,-3.02,15.0,7202,19.414769,432.637538,3.610108,Waterfront Terminal,7202,11:15:00
4,132378:305833,305833,JOHN-JUMBO to Waterfront,13,3306,Dawson & Regina,Waterfront Terminal,11:41:00,11:56:00,-3.02,15.0,7202,19.414769,432.637538,3.610108,Waterfront Terminal,7202,11:56:00


In [ ]:
route_data['total_distance'] = route_data['total_distance']/1000 # converting to km
route_data['dist_to_station'] = route_data['dist_to_station']/1000 # converting to km

In [ ]:
route_data = route_data.sort_values(['start_time', 'end_time'], ascending=True).reset_index(drop=True)

In [ ]:
route_data.head()

,trip_id,service_id,trip_headsign,route_id,shape_id,origin,destination,start_time,end_time,road_grade,total_time,total_distance,avg_speed,avg_acceleration,stop_density,has_station,dist_to_station,time_at_station
0,134287:304666,304666,EDWARD to Waterfront,5,3119,Brown & Frederica,Confederation College,06:00:00,06:09:00,-2.562,9.0,5.031,20.6084,549.5456,3.975353,NA,0.0,no_time
1,135668:304152,304152,HUDSON to Shuniah St.,7,3252,Waterfront Terminal,Shuniah & Erie,06:00:00,06:16:00,-3.904,16.0,7.655,21.308286,751.776,3.65774,NA,0.0,no_time
2,134550:305385,305385,JOHN-JUMBO to County Fair,13,3265,Waterfront Terminal,Dawson & Regina,06:00:00,06:18:00,-2.797,18.0,10.018,21.427918,427.319417,3.493711,NA,0.0,no_time
3,133771:305646,305646,MEMORIAL to Waterfront,3M,3304,City Hall Terminal,Waterfront Terminal,06:00:00,06:19:00,-1.743,19.0,7.345,17.210207,455.880828,3.948264,Waterfront Terminal,7.345,06:19:00
4,133821:305447,305447,MEMORIAL to Northwood,3M,3317,Waterfront Terminal,City Hall Terminal,06:00:00,06:19:00,-1.562,19.0,7.535,18.461333,533.667556,3.583278,City Hall Terminal,7.535,06:19:00


In [ ]:
calendar_df.loc[(calendar_df['service_id'] == 299800)]

,service_id,date,exception_type
32590,299800,20220129,1
32591,299800,20220205,1
32592,299800,20220212,1
32593,299800,20220219,1
32594,299800,20220226,1
32595,299800,20220305,1


In [ ]:
route_data.loc[route_data['trip_headsign'] == 'CROSSTOWN to Waterfront']

,trip_id,service_id,trip_headsign,route_id,shape_id,origin,destination,start_time,end_time,road_grade,total_time,total_distance,avg_speed,avg_acceleration,stop_density,has_station,dist_to_station,time_at_station
18,133878:299800,299800,CROSSTOWN to Waterfront,2,3128,Confederation College,Waterfront Terminal,06:10:00,06:39:00,-1.839,29.0,12.789,21.089822,631.360353,2.893111,Waterfront Terminal,12.789,06:39:00
105,139304:304455,304455,CROSSTOWN to Waterfront,2,3262,Confederation College,Waterfront Terminal,06:45:00,07:14:00,-1.839,29.0,10.35,19.171733,624.126476,2.898551,Waterfront Terminal,10.35,07:14:00
109,133879:300165,300165,CROSSTOWN to Waterfront,2,3128,Confederation College,Waterfront Terminal,06:50:00,07:19:00,-1.839,29.0,12.789,21.089822,631.360353,2.893111,Waterfront Terminal,12.789,07:19:00
169,139306:304088,304088,CROSSTOWN to Waterfront,2,3262,Confederation College,Waterfront Terminal,07:15:00,07:44:00,-1.839,29.0,10.35,19.171733,624.126476,2.898551,Waterfront Terminal,10.35,07:44:00
202,133880:300493,300493,CROSSTOWN to Waterfront,2,3128,Confederation College,Waterfront Terminal,07:30:00,07:59:00,-1.839,29.0,12.789,21.089822,631.360353,2.893111,Waterfront Terminal,12.789,07:59:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3018,139260:304061,304061,CROSSTOWN to Waterfront,2,3128,Confederation College,Waterfront Terminal,21:53:00,22:22:00,-1.839,29.0,12.789,21.089822,631.360353,2.893111,Waterfront Terminal,12.789,22:22:00
3109,132739:299499,299499,CROSSTOWN to Waterfront,2,3128,Confederation College,Waterfront Terminal,22:38:00,23:05:00,-1.839,27.0,12.789,21.346255,586.817341,2.893111,Waterfront Terminal,12.789,23:05:00
3110,133906:301410,301410,CROSSTOWN to Waterfront,2,3128,Confederation College,Waterfront Terminal,22:38:00,23:05:00,-1.839,27.0,12.789,21.346255,586.817341,2.893111,Waterfront Terminal,12.789,23:05:00
3111,138797:301097,301097,CROSSTOWN to Waterfront,2,3128,Confederation College,Waterfront Terminal,22:38:00,23:05:00,-1.839,27.0,12.789,21.346255,586.817341,2.893111,Waterfront Terminal,12.789,23:05:00


In [ ]:
calendar_df['month'] = pd.to_datetime(calendar_df['date'], format = date_format).dt.month_name().str.slice(stop = 3)

In [ ]:
months, dates = [], []
route_data.to_csv('route_data.csv', index = False)
shutil.move(local_path + 'route_data.csv', uni_path + 'route_data.csv')
months = list(calendar_df.month.unique())
for month in months:
  month_path = uni_path + '{}'.format(month)
  dates = list((calendar_df.loc[calendar_df['month'] == month]).date.unique())
  if not os.path.exists(month_path):
    os.mkdir(month_path)
  else:
    pass
  for date in dates:
    date_path = uni_path + '{}'.format(month) + '/' + '{}_route_data'.format(date)
    if not os.path.exists(date_path):
      os.mkdir(date_path)
    else:
      pass
    for route in routes:
      service_ids = list(calendar_df.loc[calendar_df['date'] == date, 'service_id'])
      temp_df = route_data.loc[(route_data['route_id'] == route) & (route_data['service_id'].isin(service_ids))]
      temp_df.to_csv('{}.csv'.format(route), index = False)
      shutil.move(local_path + '{}.csv'.format(route), date_path + '/' + '{}.csv'.format(route))

In [ ]:
# information retreived from https://www.timeanddate.com/weather/canada/thunder-bay/climate
weather = {}
weather['month'] = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
weather['mean_temp'] = [-14, -12, -5, 3, 9, 15, 18, 17, 12, 5, -2, -9]
weather['humidity'] = [0.72, 0.7, 0.7, 0.66, 0.68, 0.74, 0.76, 0.78, 0.8, 0.78, 0.78, 0.77]
weather['windspeed'] = [11, 11, 12, 12, 12, 11, 10, 10, 10, 11, 11, 11]
weather['visibility'] = [20, 21, 21, 23, 23, 22, 23, 23, 22, 22, 20, 20]
weather['pressure'] = [101.78, 101.90, 101.88, 101.65, 101.53, 101.35, 101.35, 101.49, 101.53, 101.55, 101.59, 101.77]
weather = pd.DataFrame.from_dict(weather, orient='index').transpose()

In [ ]:
weather.to_csv('weather_averages_thunder_bay.csv', index = False)

In [ ]:
weather_path = '/content/drive/MyDrive/Degree Project/dataset/weather'

if not os.path.exists(weather_path):
  os.mkdir(weather_path)
  shutil.move(local_path + 'weather_averages_thunder_bay.csv', weather_path + '/' + 'weather_averages_thunder_bay.csv')
  for month in weather['month']:
    temp_df = weather.loc[weather['month'] == month]
    temp_df.to_csv('{}_climate.csv'.format(month), index = False)
    shutil.move(local_path + '{}_climate.csv'.format(month), weather_path + '/' + '{}_climate.csv'.format(month))
else:
  pass

In [ ]:
routes

['5',
 '3M',
 '12',
 '3C',
 '13',
 '16',
 '7',
 '10',
 '14',
 '1',
 '11',
 '2',
 '9',
 '6',
 '3J',
 '8',
 '4']

In [ ]:
route_data.loc[route_data['route_id'] == '4', 'shape_id'].unique()

array([3221, 3219, 3215, 3218, 3302, 3224], dtype=object)